# Stage 2: Nash equilibrium

In [1]:
using JuMP, Cbc, NamedArrays
import Combinatorics

In [2]:
function get_nash_primal(score_matrix::Array)
    n = size(scores_matrix)[1]
    # for primal - P person
    m = Model()

    @variable(m, 1>= p[1:n] >=0)
    @variable(m, t)

    @constraint(m, sum(p) == 1)
    @constraint(m, min_const[k in 1:n], 
        sum(p[i] * score_matrix[i, k] for i=1:n) >= t)
    # Max min (g1, g2, ..., gn)
    @objective(m, Max, t)
    solve(m)
                
    popt = getvalue(p)
#     println(popt)
    println("Optimal Score: ", getobjectivevalue(m))
    return popt
end
;

In [12]:
function get_nash_dual(score_matrix::Array)
    n = size(scores_matrix)[2]
    # for primal - P person
    m = Model()

    @variable(m, 1>= q[1:n] >=0)
    @variable(m, t)

    @constraint(m, sum(q) == 1)
    @constraint(m, max_const[k in 1:n], 
        sum(q[i] * score_matrix[k, i] for i=1:n) <= t)
    # Max min (g1, g2, ..., gn)
    @objective(m, Min, t)
    solve(m)
                
    qopt = getvalue(q)
#     println(popt)
    println("Optimal Score: ", getobjectivevalue(m))
    return qopt
end
;

In [3]:
function get_nash_pure(score_matrix::Array)
    n = size(scores_matrix)[1]
    m = Model(solver = CbcSolver())

    @variable(m, t[1:n])
    addSOS1(m, t)
    
    for k in 1:n
        for i in 1:n
            @constraint(m, score_matrix[k, i] >= t[k])
        end
    end

    @objective(m, Max, sum(t))
    solve(m)
                
    topt = getvalue(t)
    println(topt)
    println("Optimal Score: ", getobjectivevalue(m))
    return findmax(topt)
end
;

## 1. Morra problem

### Score function

### Demo data (2 fingers Morra)

In [4]:
# calculate matrix incidences (max of mine is min of theirs or zero-sum game)
matrix_host = [2 -3; -3 4]
matrix_guest = [-2 3; 3 -4]
;

### Model

#### Host 

In [5]:
# Our strategy P
get_nash_primal(matrix_host)

LoadError: UndefVarError: scores_matrix not defined

#### Guest

In [ ]:
get_nash_primal(matrix_guest)

## 2. Chess 

### Game score function

In [4]:
# the positional advantage and the ratings difference
# higher elo, first move better
# order: 1 or 0 with 1 being first
function get_score_chess_game(player, opponent, order, elo)
    x = elo[player] - elo[opponent] + 35*(-1)^(order + 1)
    return 1/(1 + 10^(-x/400)) 
end
;

### Match score function

In [5]:
function get_match_score(team_player, team_opponent, order, elo)
    sum(get_score_chess_game(team_player[i], team_opponent[i], 
            (i + order + 1) % 2, elo) for i=1:length(team_player))
end
;

### Generate score matrices for all players
Suppose that we have 10 players, choose 4 of them to establish our team, 6 other players become our opponents

In [6]:
# score_matrix function from 2 lists
function get_score_matrix(list1::Array, list2::Array, players_dict::Array)
    # assume that team 1 plays firstly
    dim1 = length(list1)
    dim2 = length(list2)
    scores = Matrix(dim1, dim2)
    for i in 1:dim1
        for j in 1:dim2
            scores[i, j] = get_match_score(list1[i], list2[j], 1, players_dict)
        end
    end
    return scores
end

get_score_matrix (generic function with 1 method)

In [7]:
# ordered combination: return a list of nplayers obtained from a pool
function get_combinations(list_players::Array, nchosen::Int)
    return collect(combinations(list_players, nchosen))
end

get_combinations (generic function with 1 method)

In [8]:
function get_permutations(list_players::Array)
    return collect(permutations(list_players))
end

get_permutations (generic function with 1 method)

### Nash equilibrium

In [19]:
# one is enouxgh!! No of games - 1's score = 2's score
elo = [2838, 2822, 2817, 2772, 2771, 2761, 2750, 2747, 2745, 2741]
price = [1000, 900, 850, 700, 690, 650, 600, 580, 570, 560] * 0.001 

team_host = [1 3 5 6]
team_guest = [2 4 7 8]

plan_guest = [2 1 4 3] # fixed plan

N_PLAYERS = 4
N_CHOSEN = 4
N_POOL = 10
;

In [20]:
list_host = get_permutations(team_host)
list_guest = get_permutations(team_guest)
## Map to ELO
scores_matrix = get_score_matrix(list_host, list_guest, elo)
popt = get_nash_primal(scores_matrix);

Optimal Score: 2.1370614481525005


In [22]:
2.1370614481525005 - 2.1370614481525005

0.0

In [13]:
## Map to ELO
qopt = get_nash_dual(scores_matrix);

Optimal Score: 2.1370614481525005


In [28]:
findmax(popt)

(0.296820179557564,51)

In [30]:
list_host[51]

5-element Array{Int64,1}:
  5
  1
  6
  3
 10

In [18]:
get_nash_pure(scores_matrix)

Cbc3007W No integer variables - nothing to do
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.68105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
Optimal Score: 2.681054561455067


(2.681054561455067,83)

In [21]:
function f(i::Int, j::Int, k::Int)
    return get_score_chess_game(elo[team_host[i]], elo[team_guest[j]], k % 2)
end

f (generic function with 1 method)

In [22]:
n = 5
m = Model()

@variable(m, x[1:n, 1:n], Bin)
@variable(m, p[1:n])
@variable(m, q[1:n])
@variable(m, r[1:n, 1:n] <= 0)

@constraint(m, supply[k in 1:n], sum(x[k, j] for j=1:n) == 1)
@constraint(m, demand[k in 1:n], sum(x[i, k] for i=1:n) == 1)
                        
@constraint(m, mincons[j in 1:n, k in 1:n], 
    p[j] + q[k] + r[j, k] - sum(f(i, j, k) * x[i, k] for i=1:n) <= 0)
@objective(m, Max, sum(p) + sum(q) + sum(r))        

solve(m)
println(getobjectivevalue(m))
xopt = getvalue(x)
solution_x = NamedArray(Int[xopt[i, k] for i=1:n, k=1:n])
println(solution_x)
println("p: ", getvalue(p))
println("q: ", getvalue(q))
println("r: ", getvalue(r))

2.681054561455067
5×5 Named Array{Int64,2}
A ╲ B │ 1  2  3  4  5
──────┼──────────────
1     │ 0  0  0  1  0
2     │ 0  1  0  0  0
3     │ 0  0  1  0  0
4     │ 1  0  0  0  0
5     │ 0  0  0  0  1
p: [0.433968,0.50554,0.537131,0.541369,0.544171]
q: [0.0286842,0.00871989,0.0427243,0.0385311,0.000216323]
r: [0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]
